<a href="https://colab.research.google.com/github/novoforce/Exploring-Tensorflow/blob/main/2_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Data Preparation and Loading

In [2]:
(x_train,y_train), (x_test,y_test)= mnist.load_data()
print(x_train.shape,y_train.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)


In [3]:
#reshaping the tensor shape so as to feed into the neural network
x_train= x_train.reshape(-1,28*28).astype('float32') / 255.0
x_test= x_test.reshape(-1,28*28).astype('float32') / 255.0

print('reshaped tensors:> ',x_train.shape,x_test.shape,type(x_train),type(x_test))

reshaped tensors:>  (60000, 784) (10000, 784) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


#Create the AI model (Sequential API)
It is very convinient to use but not flexible. If the model to be build has **1 input and 1 output** then the **Sequential API** is the way for creating the AI model **otherwise** we have to rely on **Functional API**

In [9]:
model= keras.Sequential(
    [
     layers.Dense(512,activation='relu'),
     layers.Dense(256,activation='relu'),
     layers.Dense(10),
    ]
)

# Compile the AI model

In [11]:
#since we are not having SOFTMAX defined so using 'from_logits=True'
# Difference between "sparce categorical crossentropy" and "categorical cross entropy" is that latter expects 1-hot encoded labels
# and the previous expects integer labels
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer= keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy']
)

# Train and Evaluate the model

In [12]:
model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=2)
model.evaluate(x_test,y_test,batch_size=32,verbose=2)

Epoch 1/5
1875/1875 - 3s - loss: 0.1864 - accuracy: 0.9433
Epoch 2/5
1875/1875 - 2s - loss: 0.0790 - accuracy: 0.9759
Epoch 3/5
1875/1875 - 2s - loss: 0.0538 - accuracy: 0.9830
Epoch 4/5
1875/1875 - 2s - loss: 0.0419 - accuracy: 0.9867
Epoch 5/5
1875/1875 - 2s - loss: 0.0339 - accuracy: 0.9887
313/313 - 0s - loss: 0.0673 - accuracy: 0.9808


[0.06730663776397705, 0.9807999730110168]

# Summary of the model
Suppose we want to print the summary of the model and this can we done by passing the `keras.Input(shape=())` layers to the definition of the model and then using `model.summary()`

In [13]:
model= keras.Sequential(
    [
     
     keras.Input(shape=(28*28)),
     layers.Dense(512,activation='relu'),
     layers.Dense(256,activation='relu'),
     layers.Dense(10),
    ]
)
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________
None


# Another way of defining the sequential model
Helpful in debugging the layers as we can add `model.summary()` in between each layers and check the params info.

In [16]:
model= keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.summary()  #<----------------------- similary fashion
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(256,activation='relu'))
model.summary()  #<----------------------- similary fashion
model.add(layers.Dense(10))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
Total params: 533,248
Trainable params: 533,248
Non-trainable params: 0
_________________________________________________________________


# Functional API
This API can handle multiple input and the multiple output. So it's more flexible.

In the below example we will see the above model defined in the Functional API form


In [22]:
input= keras.Input(shape=(28*28))
x= layers.Dense(512,activation='relu')(input)
x= layers.Dense(256,activation='relu')(x)
output= layers.Dense(10,activation='softmax')(x)
model= keras.Model(inputs= input, outputs= output)

#compile the model
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer= keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy']
)

#fit the model and evaluate the model
model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=2)
model.evaluate(x_test,y_test,batch_size=32,verbose=2)

Epoch 1/5
1875/1875 - 2s - loss: 0.1865 - accuracy: 0.9427
Epoch 2/5
1875/1875 - 2s - loss: 0.0780 - accuracy: 0.9759
Epoch 3/5
1875/1875 - 2s - loss: 0.0564 - accuracy: 0.9821
Epoch 4/5
1875/1875 - 2s - loss: 0.0400 - accuracy: 0.9868
Epoch 5/5
1875/1875 - 2s - loss: 0.0332 - accuracy: 0.9896
313/313 - 0s - loss: 0.0884 - accuracy: 0.9771


[0.08839498460292816, 0.9771000146865845]

# Custom naming of the layers

In [26]:
input= keras.Input(shape=(28*28),name='input_layer')
x= layers.Dense(512,activation='relu',name='first_layer')(input)
x= layers.Dense(256,activation='relu',name='second_layer')(x)
output= layers.Dense(10,activation='softmax',name='final_layer')(x)
model= keras.Model(inputs= input, outputs= output)
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 784)]             0         
_________________________________________________________________
first_layer (Dense)          (None, 512)               401920    
_________________________________________________________________
second_layer (Dense)         (None, 256)               131328    
_________________________________________________________________
final_layer (Dense)          (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


# How to get the output of the intermediate layer ?

In [27]:
model= keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(10))

In [28]:
model= keras.Model(inputs=model.inputs,outputs=[model.layers[-2].output]) #-2 is second last layer
feature= model.predict(x_train) #we get the feature(output of the -2 layer)
print(feature.shape)

(60000, 256)


In [29]:
#suppose we want to visualize the output of layer named 'check'
model= keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(256,activation='relu',name='check'))
model.add(layers.Dense(10))

In [34]:
model= keras.Model(inputs=model.inputs,outputs=[model.get_layer('check').output]) #-2 is second last layer
feature= model.predict(x_train) #we get the feature(output of the -2 layer)
print(feature.shape)

(60000, 256)


In [35]:
#suppose we want the output of all the layers
model= keras.Model(inputs=model.inputs,outputs=[layer.output for layer in model.layers]) #-2 is second last layer
features= model.predict(x_train) #we get the feature(output of the -2 layer)

for feature in features:
    print(feature.shape)

(60000, 784)
(60000, 512)
(60000, 256)
